# Build Segment-Based Library

The notebooks use the Wildcat Creek as an example to introduce the FLDPLN model and to map the 2018 Labor Day flood inundation. [Wildcat Creek data](https://itprdkarsap.home.ku.edu/download/fldpln/wildcat_10m_3dep.zip) is available as a zip file on KU KBS-KARS server as it's too large to be put on Github. The zip file contains DEM and its derivatives, segment shapefiles, gauge Excel file and a reference flood inundation map.

In this notebook, we will first identify streams, divide them into segments and create a segment-based library for the Wildcat Creek.

## Import python packages

In [2]:
import os

# import fldpln_py and matlab packages created by the MATLAB Compiler
import fldpln_py
import matlab

# import the model module (i.e., the FLDPLN class) from the fldpln package
from fldpln.model import *

## Create a FLDPLN Model Object

To use the FLDPLN model, we first need to create a FLDPLN model object which automatically initializes the model's Python package (i.e., fldpln_py). This will activate the MATLAB Runtime and take a little bit of time.

In [11]:
# Create a FLDPLN model object
fldplnModel = FLDPLN()

## Generate Segments

The first step to create a FLDPLN library is to identify stream networks from a hydro-conditioned digital elevation model (DEM) and divide the stream networks into segments. The stream networks are first identified using a flow accumulation threshold (i.e., the strfac parameter). The stream networks are then divided into natural reaches (stream links in ArcGIS' term) which are streams between headwater and confluence pixels or between two consecutive confluence pixels. Note that pixels flow out of the DEM are treated as confluence pixels in this process. Those natural reaches are further divided into segments if the flow accumulation along its path jumps greater than or equal to another threshold (i.e., the segfac parameter). Then the segments are bisected until all segments are not longer than a certain length threshold (i.e., the seglen parameter). Note that the units for the flow accumulation thresholds are in squared miles and the unit for segment length is in miles.

The default values for the 3 parameters (i.e., strfac, segfac and seglen) are: 70 sq. miles, 25 sq. miles, and 2 miles for the libraries in KS. Note that the stream networks of Kansas FLDPLN libraries are more coarse grained than the National Water Model (NWM) reaches. This is because the KS stream networks are generated just to use available USGS gauges. If the NWM discharge is with FDLPLN libraries for the flood inundation mapping, the stream networks should to be densified.

In [ ]:
# Wildcat example
bildir = 'E:/fldpln/sites/wildcat_10m_3dep/bil'
segdir = 'E:/fldpln/sites/wildcat_10m_3dep/segs'
# Verdigris example
# bildir = 'E:/fldpln/sites/verdigris_10m/bil'
# segdir = 'E:/fldpln/sites/verdigris_10m/segs'

# Input flow direction and accumulation BIL files
fdrf = os.path.join(bildir,'fdr.bil')
facf = os.path.join(bildir,'fac.bil')

# segment parameters
strfac = 50; # flow accumulation threshold (in sq. miles) for identifying stream networks. 50 for Wildcat to include the upstream gauge. 70 for Verdigris and most KS watersheds
segfac = 25; # flow accumulation threshld (in sq. miles) for segment stream networks. 25 is the default in KS
seglen = 2; # segment length in miles. usually is the SQRT of sgefac. 2 for Wildcat and 5 for Verdigris and others in KS

# generate segments
fldplnModel.GenerateSegments(fdrf, facf, strfac, segfac, seglen, segdir)

# write FSP information as a MAT file for use in generating stream order and segment info as a CSV files
seg_list = []; # [] for all the segments or a column vector (i.e., seg_list = [1;2;3])for selected segments
fldplnModel.WriteSegmentFspCsvFiles(bildir, segdir, seg_list, segdir, 'mat')

## Select Segments and Create Spatial Mask

We don't need to generate FLDPLN library for every segment identified. And typically we want to exclude large waterbodies such as reservoirs/lakes and the segments that within them from the modeling.

### Generate Segment Shapefile 

Here we create a segment shapefile using the ArcGIS Pro's Stream to Features tool with str_segid.bil and fdr.bil as the input rasters. The segment shapefiles can then be used to exam and select a subset of segments for running the FLDPLN model.

Note that the Raster to Polyline tool in ArcGIS Pro doesn't work properly in some cases. In the figure below, the left stream polyline is generated by the Raster to Polyline tool, and the right stream polyline is generated by the Stream to Feature tool. The left polyline is erroneous. Also note that the Stream to Features tool connects upstream segments at the downstream confluence pixel.

![](./images/stream2feature.PNG)

### Select Segments

Using the shapefiles generated from the above step, we can exam the stream networks and segments and see whether they are appropriate, and may re-generate them if necessary.

With the stream networks and segments, we can select the segments in a sub-watershed or any subset of segments to create a FLDPLN library. Take a note on the field that stores the segment ID in the shapefile, and we may also add a new field to store the max stage modelled in the shapefile. The subset of selected segments can be exported to a new shapefile, which can then be used as one of the parameter in the next step.

### Prepare Spatial Mask

Areas surrounding large waterbodies such as reservoirs and lakes should be masked out and NOT included before running the FLDPLN model as their potential flood inundation is different from river and is is caused by their water level which is usually controlled by human or USACE in the United States. Based on Kansas experience, for most lakes/reservoirs, the inundation extent of their 75 percentile of water surface elevation is good mask to use. All the segments that intersect with the mask should be removed from the segment shapefile.  

Note that the mask raster must be in the BIL format. When a mask BIL raster is provided as the input parameter when building a FLDPLN  library, those cells within the mask, including both stream cells and floodplain cells, are remove from the modeling. 

## Create Segment-Based Library

With the segments decided and mask created, we can create a FLDPLN library for the segments. **Note that this is the most time consuming step in building a FLDPLN libraries, especially when the max stage modelled is high.**

### Create a library for selected segments

With the segment shapefiles, we need to specify the segment ID field/column in the shapefile.  All the selected segments by default have the same max. flood stage as specified by model parameter mxht. We can also specify a second column in the shapefile to define different max. flood stages for individual segment.

In [ ]:
# set input and output folders
# Wildcat example
bildir = 'E:/fldpln/sites/wildcat_10m_3dep/bil' 
segdir = 'E:/fldpln/sites/wildcat_10m_3dep/segs'
libdir = 'E:/fldpln/sites/wildcat_10m_3dep/rawlib' # raw segment-based library

#
# Spatial mask and segment shapefile
#
# Applying a spatial mask if provided. The mask removes the waterbodies and potentially inundated areas by them from the FLDPLN model
filmskfile = '' # no spatial mask for Wildcat. Set to '' for no mask
# filmskfile = 'E:/fldpln/sites/verdigris_10m/bil/fil_masked.bil'

# The subset of segments and their modelled max stage are defined by a dictionary of 3 keys: 'file', 'segid_field', and 'seg_fldmx_field', 
# where 'file' is the path to the shapefile, 'segid_field' is the field name for segment ID, and 'seg_fldmx_field' is the field name for the max stage.
# set the shapefile path
# segshpfile = {'file': ''} # all the segments will be used and they use the fldmx
segshpfile = {'file':'E:/fldpln/sites/wildcat_10m_3dep/segment_shapefiles/wildcat_segments.shp'} # 16 Wildcat creek segments
# segshpfile = {'file':'E:/fldpln/sites/verdigris_10m/segs_5mi/dam_break_segments.shp'} # verdigris
# set the two additional keys if a segment shapefile is provided
segshpfile['segid_field'] = 'grid_code' # field name for segment ID
segshpfile['seg_fldmx_field'] = '' # set to '' when all the segments use the same fldmx. Otherwise specify a field in the shapefile

#
# Set FLDPLN model parameters
#
fldmn = 0.01 # typically set to 1 centimeter or 0.0328084 foot deoends on DEM's vertical unit
fldmx = 15 # max. stage modeled in DEM's vertical unit. Both wildcat_10m_3dep and verdigris DEM vertical unit is in meters
dh = 1 # vertical step size in DEM's vertical unit
mxht = 0 # max(dem+flood height) to cease flooding. enter 0 for no cap height

#
# FLDPLN model run parameters
#
# model run memory type: choose one from {'hd', 'ram0', 'ram'}. 
mtype = 'ram' # choose either 'ram' (machine has RAM >= 64G) or 'hd' (uses least RAM)

# parallelization parameters can be defined by a dictionary of 3 keys: 'type', 'numcores', and 'worker_type', 
# where 'type' is the parallelization type, 'numcores' is the number of cores, and 'worker_type' is the type of worker.
# set the parallelization type. Choose one from {'none', 'parfor', 'parfeval'} but 'parfeval' is preferred
para = {'type': 'parfeval'}
# set parallelization parameter keys if the model is run in parallel (i.e., para['type'] is not 'none')
# set the number of cores, and if not set the max number of cores will be used
para['numcores'] = 6 # Comment out to use all the cores available
# set the worker type. Choose one from {'Threads', 'Processes'}. 'Threads' is not supported by compiled Python package!
para['worker_type'] = 'Processes'

# Run the model to create a segment-based library
fldplnModel.CreateSegmentLibrary(bildir, segdir, filmskfile, segshpfile, fldmn, fldmx, dh, mxht, libdir, mtype, para)

## Format Segment-Based Library for Tiling

The raw segment-based library built in the previous step doesn't have map coordinates. Here we reformat the raw library and make it ready for tiling.

In [ ]:
# BIL file directory
bildir = 'E:/fldpln/sites/wildcat_10m_3dep/bil' # Wildcat
# segment file directory
segdir = 'E:/fldpln/sites/wildcat_10m_3dep/segs'
# raw segment library dir
libdir = 'E:/fldpln/sites/wildcat_10m_3dep/rawlib'

# Outputs:
# Output library folder
dirout = 'E:/fldpln/sites/wildcat_10m_3dep/seglib' # reformatted library for tiling and mapping

# Format raw segment library
fldplnModel.FormatSegmentLibrary(bildir, segdir, libdir, dirout)

## Assign Stream Order to Segments

Stream orders are used while interpolating the FSP depth of flow (DOF) from gauge stages. Interpolation is performed from low order to high order streams. The general rules for assigning stream order are:
* A stream (or more accurately a reach) consists of several connected segments which flows out of the DEM or into another stream at a confluence.
* Lower order streams are main streams where FSP DOF are first interpolated first and confluences on the main streams may serve as ggauges when interpolating in-flow high order tributary streams.
* Stream order must be unique in a library. No two streams can have the same order even if they are in different sub-stream networks in the library

The above requirements are necessary to make sure the interpolation on FSP DOF is carried out correctly. Stream orders can be assigned either manually of automatically. The following code shows an automatic approach of assigning stream orders and it's new function and still under testing and verification.

In [ ]:
# BIL file directory
bildir = 'E:/fldpln/sites/wildcat_10m_3dep/bil' # Wildcat
# segment file directory
segdir = 'E:/fldpln/sites/wildcat_10m_3dep/segs'
# segment shapefile
segshp = 'E:/fldpln/sites/wildcat_10m_3dep/segment_shapefiles/wildcat_segments.shp' 

# Generate stream order. Four files are created in the segdir folder by the method: 
# network_slopes4stream_ordering.mat and network_slopes4stream_ordering.xlsx file
# Two shapefiles with subnetwwork ID and stream order within the subnetwork

fldplnModel.GenerateStreamOrder(bildir, segdir, segshp)

## Terminate the FLDPLN Model

This is done by delete the FLDPLN model object.

In [10]:
del fldplnModel
# or
# fldplnModel = None

## Issues and Questions

* Messages displayed in the FLDPLN Python package are NOT shown in this notebook but running the code cells as a python script the messages are shown just fine.
